In [1]:
!pip install langchain-openai langchain-community huggingface_hub PyPDF2 langchain-huggingface faiss-cpu langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00


In [2]:
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from huggingface_hub import login
from langchain_groq import ChatGroq
import os

In [4]:
load_dotenv()
login(os.getenv("HUGGINGFACEHUB_API_TOKEN"))

In [5]:
from google.colab import files
uploaded = files.upload()

Saving Custom_Formatted_Events_1900_to_2025.pdf to Custom_Formatted_Events_1900_to_2025.pdf


In [6]:
pdf_paths = [f"/content/{filename}" for filename in uploaded.keys()]
print(f"selected PDFs: {pdf_paths}")

selected PDFs: ['/content/Custom_Formatted_Events_1900_to_2025.pdf']


In [7]:
def get_pdf_text(pdf_docs):
  '''
  loop over all the pdf files and concatenate
  the content in a single string
  '''
  text = ""
  for pdf in pdf_paths:
    pdf_reader = PdfReader(pdf)
    for page in pdf_reader.pages:
      page_text = page.extract_text()
      if page_text:
        text += page_text
    return text


In [8]:
raw_text = get_pdf_text(pdf_paths)
print("✅ Extracted text length:", len(raw_text))

✅ Extracted text length: 25396


In [9]:
def get_text_chunks(raw_text):
  '''
  takes a single string of text and returns a list
  of text strings that can be fed to vector database
  '''
  text_splitter = CharacterTextSplitter(
      separator="\n",
      chunk_size=1000,
      chunk_overlap=200,
      length_function=len
  )
  chunks = text_splitter.split_text(raw_text)
  return chunks


In [10]:
text_chunks = get_text_chunks(raw_text)
print("✅ Number of text chunks:", len(text_chunks))

✅ Number of text chunks: 32


In [11]:
def get_vectorstore(text_chunks):
  '''
  creat a FAISS vectorstore from the text chunks using embeddings
  '''
  embeddings = HuggingFaceEmbeddings(
      model_name="hkunlp/instructor-xl",
      model_kwargs={"device": "cpu"}
  )
  vectorstore = FAISS.from_texts(
      texts=text_chunks,
      embedding=embeddings
  )
  return vectorstore


In [12]:
vectorstore = get_vectorstore(text_chunks)
print("✅ vectorstore created")

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

✅ vectorstore created


In [13]:
def get_conversation_chain(vectorstore):
    '''
    Creates a conversational retrieval chain using OpenAI's mode
    '''
   #llm = ChatOpenAI(
     # model_name= "gpt-4o-mini",
     # temperature=0.7,
     # request_timeout=30
    llm = ChatGroq(
         model_name="llama-3.3-70b-versatile",
         temperature=0.7,
         request_timeout=30,
         api_key="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"
    )
    memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=True
    )
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain


In [14]:
conversation = get_conversation_chain(vectorstore)

/tmp/ipython-input-13-4103226084.py:15: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [15]:
def chat_with_pdf(question):
    response = conversation({'question': question})
    for i, msg in enumerate(response['chat_history']):
      sender = "Query````````````1                                        " if i % 2 == 0 else "Timeline"
      print(f"{sender}: {msg.content}\n")

In [16]:
chat_with_pdf("Events occured in 1985?")
chat_with_pdf("Timeline of Europian History?")
chat_with_pdf("What happened in 1938?")

/tmp/ipython-input-15-2309804136.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = conversation({'question': question})


Query````````````1                                        : Events occured in 1985?

Timeline: In 1985, the following events occurred:

1. Cultural renaissance begins in Europe.
2. Landmark ruling passed in India on gender equality.
3. Landmark ruling passed in China on human rights.

Query````````````1                                        : Events occured in 1985?

Timeline: In 1985, the following events occurred:

1. Cultural renaissance begins in Europe.
2. Landmark ruling passed in India on gender equality.
3. Landmark ruling passed in China on human rights.

Query````````````1                                        : Timeline of Europian History?

Timeline: Based on the provided context, here is a limited timeline of European history:

* 1942: Disaster strikes in London, affecting thousands.
* 1943: 
  * Cultural renaissance begins in Europe.
  * Scientific breakthrough in communication revolutionizes global connectivity.
  * Massive protest erupts in Paris over economic reform.